# **<span style="color: Purple;"> Hourly ERA-5 Reanalysis Soundings</span>**
## **<span style="color: orange;">Created by: John Farash - Advised by: Kevin Tyle:</span>**
***


### **This notebook displays ERA-5 Renanalysis soundings for a maximum of 1 week**

#### **Documentation of Panel interactivity (https://panel.holoviz.org)**
#### **Documentation of SounderPy (https://kylejgillett.github.io/sounderpy/)**
***

### **Import everything we need**

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
from metpy.units import units
import metpy.calc as mpcalc
from scipy import ndimage  
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl
import metpy
from metpy.plots import ctables
import sounderpy as spy
import panel as pn 
import warnings
warnings.filterwarnings('ignore')
pn.extension()

## **Date Input**
### **Input a start and end date using the cells below**
***

In [ ]:
start1 = pn.widgets.DatePicker(name='Start Date')
end1 = pn.widgets.DatePicker(name='End Date')
pn.Row(start1,end1, height=400)

## **Time Input**
### **Input a start and end time**
***

In [ ]:
starttime1 = pn.widgets.Select(name='Start Time UTC', options= ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])

endtime1 = pn.widgets.Select(name='End Time UTC', options=['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])

layout = pn.Column(starttime1, endtime1)
layout.servable()

## **Run this cell to check for a 1 week time span and that the date and time inputs are sucsessful**
***

In [ ]:
start_time_str = starttime1.value 
end_time_str = endtime1.value     

time_start = dt.strptime(start_time_str, "%H").time()
time_end = dt.strptime(end_time_str, "%H").time()

start_datetime = dt.combine(start1.value, time_start)
end_datetime = dt.combine(end1.value, time_end)


if end_datetime < start_datetime:
    raise ValueError("**Warning:** End date and time cannot be before start date and time!")

datelist = pd.date_range(start_datetime, end_datetime,freq="6h")
if len(datelist) > 28:
    raise ValueError("WARNING!: The date range exceeds one week, please go back to the date inputs and make the time range less than a week. ")
else:
    print("Sucsess!")

dateList=datelist
dateList = dateList.strftime('%Y,%d,%m,%H')
dateList

## **Input sounding analysis location or select location by state or territory**
***

In [ ]:
# Define a dictionary of cities with their coordinates
city_coords = {
    '-ALABAMA-':(None,None),
    'Birmingham, AL': (33.5186, -86.8104),
    'Dothan, AL': (31.2232, -85.3905),
    'Florence, AL': (34.7998, -87.6773),
    'Gadsden, AL': (34.0143, -86.0066),
    'Hoover, AL': (33.4054, -86.8114),
    'Huntsville, AL': (34.7304, -86.5861),
    'Mobile, AL': (30.6954, -88.0399),
    'Montgomery, AL': (32.3792, -86.3077),
    'Selma, AL': (32.4074, -87.0211),
    'Tuscaloosa, AL': (33.2098, -87.5692),
    '':(None,None),
 
    '-ALASKA-':(None,None),
    'Anchorage, AK': (61.2181, -149.9003),
    'Barrow (Utqiaġvik), AK': (71.2906, -156.7886),
    'Bethel, AK': (60.7922, -161.7558),
    'Fairbanks, AK': (64.8378, -147.7164),
    'Homer, AK': (59.6425, -151.5483),
    'Juneau, AK': (58.3019, -134.4197),
    'Kenai, AK': (60.5544, -151.2583),
    'Ketchikan, AK': (55.3422, -131.6461),
    'Kodiak, AK': (57.7890, -152.4072),
    'Nome, AK': (64.5011, -165.4064),
    'Palmer, AK': (61.6008, -149.1128),
    'Seward, AK': (60.1042, -149.4422),
    'Sitka, AK': (57.0531, -135.3300),
    'Soldotna, AK': (60.4870, -151.0583),
    'Valdez, AK': (61.1308, -146.3483),
    'Wasilla, AK': (61.5814, -149.4394),
    'Wrangell, AK': (56.4708, -132.3767),
    '':(None,None),
 

    '-ARIZONA-':(None,None),
    'Chandler, AZ': (33.3062, -111.8413),
    'Flagstaff, AZ': (35.1983, -111.6513),
    'Gilbert, AZ': (33.3528, -111.7890),
    'Glendale, AZ': (33.5387, -112.1860),
    'Mesa, AZ': (33.4152, -111.8315),
    'Phoenix, AZ': (33.4484, -112.0740),
    'Prescott, AZ': (34.5400, -112.4685),
    'Scottsdale, AZ': (33.4942, -111.9261),
    'Tempe, AZ': (33.4255, -111.9400),
    'Tucson, AZ': (32.2226, -110.9747),
    'Yuma, AZ': (32.6927, -114.6277),
    '':(None,None),

    '-ARKANSAS-':(None,None),
    'Bentonville, AR': (36.3729, -94.2088),
    'Conway, AR': (35.0887, -92.4421),
    'Fayetteville, AR': (36.0822, -94.1719),
    'Fort Smith, AR': (35.3859, -94.3985),
    'Hot Springs, AR': (34.5037, -93.0552),
    'Jonesboro, AR': (35.8423, -90.7043),
    'Little Rock, AR': (34.7465, -92.2896),
    'Pine Bluff, AR': (34.2284, -92.0032),
    'Rogers, AR': (36.3320, -94.1185),
    'Springdale, AR': (36.1867, -94.1288),
    '':(None,None),

    '-CALIFORNIA-':(None,None),
    'Anaheim, CA': (33.8366, -117.9143),
    'Bakersfield, CA': (35.3733, -119.0187),
    'Berkeley, CA': (37.8716, -122.2727),
    'Bridgeport, CA': (38.2558, -119.2318),
    'Death Valley, CA': (36.5054, -116.9325),
    'El Centro, CA': (32.7920, -115.5631),
    'Eureka, CA': (40.8021, -124.1637),
    'Fresno, CA': (36.7378, -119.7871),
    'Independence, CA': (36.8022, -118.2009),
    'Long Beach, CA': (33.7701, -118.1937),
    'Los Angeles, CA': (34.0522, -118.2437),
    'Oakland, CA': (37.8044, -122.2712),
    'Redding, CA': (40.5865, -122.3917),
    'Sacramento, CA': (38.5816, -121.4944),
    'San Diego, CA': (32.7157, -117.1611),
    'San Francisco, CA': (37.7749, -122.4194),
    'San Jose, CA': (37.3382, -121.8863),
    'Santa Barbara, CA': (34.4208, -119.6982),
    'Stockton, CA': (37.9577, -121.2908),
    'Yreka, CA': (41.7356, -122.6347),
    '':(None,None),

    '-COLORADO-':(None,None),
    'Aspen, CO': (39.1911, -106.8175),
    'Boulder, CO': (40.0150, -105.2705),
    'Colorado Springs, CO': (38.8339, -104.8214),
    'Denver, CO': (39.7392, -104.9903),
    'Durango, CO': (37.2753, -107.8801),
    'Fort Collins, CO': (40.5853, -105.0844),
    'Grand Junction, CO': (39.0639, -108.5506),
    'Gunnison, CO': (38.5458, -106.9253),
    'Pueblo, CO': (38.2544, -104.6091),
    'Steamboat Springs, CO': (40.4848, -106.8317),
    'Vail, CO': (39.6403, -106.3742),
    '':(None,None),

    '-CONNECTICUT-':(None,None),
    'Bridgeport, CT': (41.1792, -73.1894),
    'Danbury, CT': (41.3948, -73.4540),
    'Hartford, CT': (41.7658, -72.6734),
    'New Haven, CT': (41.3083, -72.9279),
    'New London, CT': (41.3557, -72.0995),
    'Norwalk, CT': (41.1177, -73.4082),
    'Stamford, CT': (41.0534, -73.5387),
    'Waterbury, CT': (41.5582, -73.0515),
    '':(None,None),

    '-DELAWARE-':(None,None),
    'Dover, DE': (39.1582, -75.5244),
    'Middletown, DE': (39.4497, -75.7161),
    'New Castle, DE': (39.6625, -75.5642),
    'Wilmington, DE': (39.7391, -75.5398),

    '-FLORIDA-':(None,None),
    'Clearwater, FL': (27.9659, -82.8001),
    'Fort Lauderdale, FL': (26.1223, -80.1434),
    'Fort Myers, FL': (26.6406, -81.8723),
    'Gainesville, FL': (29.6516, -82.3248),
    'Jacksonville, FL': (30.3322, -81.6556),
    'Maiami, FL': (25.7617, -80.1918),
    'Naples, FL': (26.1420, -81.7948),
    'Orlando, FL': (28.5383, -81.3792),
    'Pensacola, FL': (30.4213, -87.2169),
    'St. Petersburg, FL': (27.7676, -82.6403),
    'Tampa, FL': (27.9506, -82.4572),
    'West Palm Beach, FL': (26.7153, -80.0534),

    '-GEORGIA-':(None,None),
    'Atlanta, GA': (33.748995, -84.387982),
    'Athens, GA': (33.9519, -83.3576),
    'Augusta, GA': (33.4735, -82.0105),
    'Columbus, GA': (32.4609, -84.9877),
    'Macon, GA': (32.8407, -83.6324),
    'Marietta, GA': (33.9526, -84.5499),
    'Roswell, GA': (34.0236, -84.3616),
    'Savannah, GA': (32.0836, -81.0998),
    'Smyrna, GA': (33.8834, -84.5145),
    'Valdosta, GA': (30.8327, -83.2785),

    '-HAWAII-':(None,None),
    'Honolulu, HI': (21.3069, -157.8583),
    'Hilo, HI': (19.7299, -155.0915),
    'Kailua, HI': (21.3995, -157.7394),
    'Kaneohe, HI': (21.4191, -157.8086),
    'Lihue, HI': (21.9784, -159.3710),
    'Makakilo, HI': (21.3546, -158.0831),
    'Pearl City, HI': (21.4054, -157.9747),
    'Waianae, HI': (21.4463, -158.1806),
   
    '-IDAHO-':(None,None),
    'Boise, ID': (43.6150, -116.2023),
    'Coeur d\'Alene, ID': (47.6777, -116.7805),
    'Idaho Falls, ID': (43.4917, -112.0333),
    'Moscow, ID': (46.7310, -116.9970),
    'Nampa, ID': (43.5407, -116.5635),
    'Pocatello, ID': (42.8713, -112.4455),
    'Twin Falls, ID': (42.5629, -114.4609),
    'Rexburg, ID': (43.6695, -111.7893),
    'Meridian, ID': (43.6126, -116.3915),
    'Post Falls, ID': (47.7124, -116.9519),

    '-ILLINOIS-':(None,None),
    'Aurora, IL': (41.7606, -88.3201),
    'Bloomington, IL': (40.4842, -88.9937),
    'Carbondale, IL': (37.7273, -89.2168),
    'Champaign, IL': (40.1164, -88.2434),
    'Chicago, IL': (41.8781, -87.6298),
    'Decatur, IL': (39.8403, -88.9548),
    'Elgin, IL': (42.0359, -88.2812),
    'Joliet, IL': (41.5250, -88.0817),
    'Naperville, IL': (41.7489, -88.1535),
    'Peoria, IL': (40.6936, -89.5890),
    'Rockford, IL': (42.2711, -89.0940),
    'Skokie, IL': (42.0334, -87.7322),
    'Springfield, IL': (39.7983, -89.6545),
    'Waukegan, IL': (42.3636, -87.8448),

    
    '-INDIANA-':(None,None),
    'Anderson, IN': (40.1053, -85.6803),
    'Bloomington, IN': (39.1653, -86.5264),
    'Charlestown, IN': (38.4464, -85.6740),
    'Columbus, IN': (39.2010, -85.9214),
    'Evansville, IN': (37.9715, -87.5711),
    'Fort Wayne, IN': (41.0793, -85.1394),
    'Gary, IN': (41.5934, -87.3462),
    'Indianapolis, IN': (39.7684, -86.1580),
    'Kokomo, IN': (40.4864, -86.1336),
    'Lafayette, IN': (40.4259, -86.8753),
    'Muncie, IN': (40.1934, -85.3864),
    'Noblesville, IN': (40.0451, -86.0089),
    'South Bend, IN': (41.6764, -86.2500),
    'Terre Haute, IN': (39.4667, -87.4139),
    'West Lafayette, IN': (40.4259, -86.9081),

    '-IOWA-':(None,None),
    'Ankeny, IA': (41.7315, -93.6002),
    'Bettendorf, IA': (41.5772, -90.5092),
    'Council Bluffs, IA': (41.2619, -95.8608),
    'Des Moines, IA': (41.5868, -93.6250),
    'Dubuque, IA': (42.5006, -90.6646),
    'Iowa City, IA': (41.6611, -91.5302),
    'Marshalltown, IA': (42.0480, -92.9010),
    'Mason City, IA': (43.1535, -93.2011),
    'Newton, IA': (41.7005, -93.0332),
    'Sioux City, IA': (42.4997, -96.4003),
    'Waterloo, IA': (42.4928, -92.3426),

    '-KANSAS-':(None,None),
    'Augusta, KS': (37.6877, -97.0245),
    'Emporia, KS': (38.4055, -96.1883),
    'Garden City, KS': (37.9775, -100.8715),
    'Johnson City, KS': (37.9389, -101.7120),
    'Kansas City, KS': (39.1141, -94.6275),
    'Lawrence, KS': (38.9717, -95.2353),
    'Leavenworth, KS': (39.3134, -94.9291),
    'Manhattan, KS': (39.1804, -96.5717),
    'Newton, KS': (38.0402, -97.3455),
    'Overland Park, KS': (38.9822, -94.6708),
    'Salina, KS': (38.8403, -97.6116),
    'Tonganoxie, KS': (39.1365, -95.0104),
    'Topeka, KS': (39.0558, -95.6891),
    'Wichita, KS': (37.6872, -97.3301),

    '-KENTUCKY-':(None,None),
    'Ashland, KY': (38.4747, -82.6305),
    'Bowling Green, KY': (36.9685, -86.4467),
    'Covington, KY': (39.0837, -84.5082),
    'Elizabethtown, KY': (37.7130, -85.8595),
    'Florence, KY': (38.9986, -84.6359),
    'Frankfort, KY': (38.2009, -84.8733),
    'Hopkinsville, KY': (36.8599, -87.4877),
    'Lexington, KY': (38.0406, -84.5037),
    'Louisville, KY': (38.2527, -85.7585),
    'Madisonville, KY': (37.3313, -87.4989),
    'Monticello, KY': (36.9264, -84.8773),
    'Owensboro, KY': (37.7742, -87.1113),
    'Paducah, KY': (37.0834, -88.6137),
    'Richmond, KY': (37.7361, -84.2700),
    'Somerset, KY': (37.0922, -84.6065),
    'Walton, KY': (38.8329, -84.5984),

    '-LOUISIANA-':(None,None),
    'Abbeville, LA': (29.9692, -92.1251),
    'Alexandria, LA': (31.3113, -92.4451),
    'Baton Rouge, LA': (30.4583, -91.1403),
    'Bossier City, LA': (32.5155, -93.7321),
    'Covington, LA': (30.4754, -90.1001),
    'Denham Springs, LA': (30.4120, -90.9452),
    'Houma, LA': (29.5930, -90.7215),
    'La Place, LA': (30.0701, -90.5093),
    'Lafayette, LA': (30.2241, -92.0198),
    'Lake Charles, LA': (30.2266, -93.2176),
    'Mandeville, LA': (30.3751, -90.0745),
    'Metairie, LA': (30.0041, -90.1529),
    'Monroe, LA': (32.5093, -92.1193),
    'New Iberia, LA': (30.0032, -91.8055),
    'New Orleans, LA': (29.9511, -90.0715),
    'Shreveport, LA': (32.5252, -93.7502),
    'Slidell, LA': (30.2760, -89.7778),
    'West Monroe, LA': (32.5107, -92.1848),

    '-MAINE-':(None,None),
    'Augusta, ME': (44.3106, -69.7795),
    'Bangor, ME': (44.8012, -68.7778),
    'Bath, ME': (43.9101, -69.8206),
    'Biddeford, ME': (43.4890, -70.4547),
    'Brunswick, ME': (43.9148, -69.9658),
    'Caribou, ME': (46.8760, -68.0032),
    'Lewiston, ME': (44.1007, -70.2148),
    'Portland, ME': (43.6615, -70.2553),
    'Saco, ME': (43.5023, -70.4422),
    'Sanford, ME': (43.4280, -70.7745),
    'Waterville, ME': (44.5468, -69.6275),

    '-MARYLAND-':(None,None),
    'Annapolis, MD': (38.9784, -76.4922),
    'Baltimore, MD': (39.2904, -76.6122),
    'Bethesda, MD': (38.9806, -77.1002),
    'Bowie, MD': (39.0068, -76.7990),
    'College Park, MD': (38.9794, -76.9378),
    'Frederick, MD': (39.4143, -77.4105),
    'Hagerstown, MD': (39.6418, -77.7199),
    'Laurel, MD': (39.0997, -76.8446),
    'Piney Point, MD': (38.1905, -76.5207),
    'Rockville, MD': (39.0836, -77.1528),
    'Silver Spring, MD': (38.9907, -77.0261),
    'Takoma Park, MD': (38.9807, -77.0127),

    '-MASSACHUSETTES-':(None,None),
    'Boston, MA': (42.3601, -71.0589),
    'Brockton, MA': (42.0834, -71.0184),
    'Cambridge, MA': (42.3736, -71.1097),
    'Cape Cod, MA': (41.7000, -70.3000),  
    'Fall River, MA': (41.7015, -71.1552),
    'Framingham, MA': (42.2793, -71.4162),
    'Gloucester, MA': (42.6150, -70.6613),
    'Lawrence, MA': (42.7070, -71.1631),
    'Lynn, MA': (42.4668, -70.9495),
    'Lowell, MA': (42.6334, -71.3162),
    'New Bedford, MA': (41.6362, -70.9313),
    'North Adams, MA': (42.7012, -73.1085),
    'Pittsfield, MA': (42.4505, -73.2454),
    'Quincy, MA': (42.2529, -71.0023),
    'Springfield, MA': (42.1015, -72.5898),
    'Worcester, MA': (42.2626, -71.8023),

    '-MICHIGAN-':(None,None),
    'Ann Arbor, MI': (42.2808, -83.7430),
    'Battle Creek, MI': (42.3211, -85.1798),
    'Bay City, MI': (43.5895, -83.8883),
    'Detroit, MI': (42.3314, -83.0458),
    'Flint, MI': (43.0125, -83.6875),
    'Grand Blanc, MI': (42.9361, -83.6221),
    'Grand Rapids, MI': (42.9634, -85.6681),
    'Holland, MI': (42.7875, -86.1086),
    'Kalamazoo, MI': (42.2917, -85.5872),
    'Lansing, MI': (42.7325, -84.5555),
    'Livonia, MI': (42.3684, -83.3527),
    'Muskegon, MI': (43.2348, -86.2484),
    'Pontiac, MI': (42.6389, -83.2915),
    'Saginaw, MI': (43.4195, -83.9502),
    'Sterling Heights, MI': (42.5803, -83.0302),
    'Troy, MI': (42.6064, -83.1499),
    'Waterford, MI': (42.6267, -83.3676),
    'Westland, MI': (42.3071, -83.4000),
    'Wyoming, MI': (42.9345, -85.7192),

    '-MISSISSIPPI-':(None,None),
    'Biloxi, MS': (30.3674, -88.8792),
    'Brandon, MS': (32.3852, -90.0021),
    'Gulfport, MS': (30.3674, -89.0928),
    'Hattiesburg, MS': (31.3271, -89.2903),
    'Jackson, MS': (32.2988, -90.1848),
    'Laurel, MS': (31.6894, -89.1400),
    'Meridian, MS': (32.3643, -88.7032),
    'Mississippi City, MS': (30.3753, -88.6712),
    'Oxford, MS': (34.3661, -89.5192),
    'Pearl, MS': (32.2992, -90.1500),
    'Picayune, MS': (30.5260, -89.1854),
    'Southaven, MS': (34.9870, -90.0000),
    'Vicksburg, MS': (32.3526, -90.8779),

    '-MISSOURI-':(None,None),
    'Columbia, MO': (38.9517, -92.3341),
    'Independence, MO': (39.0911, -94.4155),
    'Kansas City, MO': (39.0997, -94.5786),
    'Lee\'s Summit, MO': (38.9108, -94.3820),
    'St. Charles, MO': (38.7810, -90.4998),
    'St. Louis, MO': (38.6270, -90.1994),
    'Saint Joseph, MO': (39.7671, -94.8466),
    'Springfield, MO': (37.2153, -93.2983),

    '-MONTANA-':(None,None),
    'Billings, MT': (45.7833, -108.5007),
    'Bozeman, MT': (45.6796, -111.0383),
    'Butte, MT': (46.0037, -112.5348),
    'Cut Bank, MT': (48.6113, -112.3599),
    'Glendive, MT': (47.1415, -104.7121),
    'Great Falls, MT': (47.5000, -111.3000),
    'Havre, MT': (48.5547, -109.6775),
    'Helena, MT': (46.5958, -112.0270),
    'Kalispell, MT': (48.1950, -114.3140),
    'Lewistown, MT': (47.0683, -109.4273),
    'Livingston, MT': (45.6617, -110.5619),
    'Miles City, MT': (46.4123, -105.8499),
    'Missoula, MT': (46.8740, -113.9962),
    'Red Lodge, MT': (45.1830, -109.2470),
    'Billings, MT': (45.7833, -108.5007),

    '-NEBRASKA-':(None,None),
    'Beatrice, NE': (40.2650, -96.7467),
    'Bellevue, NE': (41.1536, -95.9750),
    'Columbus, NE': (41.4351, -98.0402),
    'Fremont, NE': (41.4337, -96.5000),
    'Grand Island, NE': (40.9255, -98.3420),
    'Kearney, NE': (40.7003, -98.0016),
    'Lincoln, NE': (40.8136, -96.7026),
    'Norfolk, NE': (41.9983, -98.3981),
    'Omaha, NE': (41.2565, -95.9345),
    'Scottsbluff, NE': (41.8667, -103.6673),
    'York, NE': (40.8764, -97.6002),

    '-NEVADA-':(None,None),
    'Boulder City, NV': (35.9783, -114.8330),
    'Carson City, NV': (39.1638, -119.7674),
    'Elko, NV': (40.8324, -115.7661),
    'Henderson, NV': (36.0395, -114.9817),
    'Las Vegas, NV': (36.1699, -115.1398),
    'Lincoln, NV': (36.4766, -114.3524),
    'Reno, NV': (39.5296, -119.8138),
    'Sparks, NV': (39.5349, -119.7527),
    'Sun Valley, NV': (39.6136, -119.7765),
    'West Wendover, NV': (40.7380, -114.0667),

    '-NEW HAMPSHIRE-':(None,None),
    'Concord, NH': (43.2081, -71.5375),
    'Durham, NH': (43.1394, -70.9283),
    'Hampton, NH': (42.9334, -70.8470),
    'Keene, NH': (42.9334, -72.2801),
    'Laconia, NH': (43.5263, -71.4637),
    'Lebanon, NH': (43.6407, -72.2515),
    'Merrimack, NH': (42.8746, -71.5149),
    'Manchester, NH': (42.9956, -71.4547),
    'Mt. Washington, NH': (44.2706, -71.3032),
    'Nashua, NH': (42.7654, -71.4677),
    'Portsmouth, NH': (43.0710, -70.7626),

    '-NEW JERSEY-':(None,None),
    'Atlantic City, NJ': (39.3643, -74.4229),
    'Bayonne, NJ': (40.6682, -74.1143),
    'Camden, NJ': (39.9259, -75.1193),
    'East Orange, NJ': (40.7673, -74.2049),
    'Elizabeth, NJ': (40.6636, -74.2107),
    'Ewing, NJ': (40.2684, -74.7982),
    'Hackensack, NJ': (40.8859, -74.0435),
    'Hoboken, NJ': (40.7431, -74.0324),
    'Jersey City, NJ': (40.7178, -74.0431),
    'New Brunswick, NJ': (40.4862, -74.4487),
    'Newark, NJ': (40.7357, -74.1724),
    'Passaic, NJ': (40.8584, -74.1300),
    'Paterson, NJ': (40.9168, -74.1718),
    'Perth Amboy, NJ': (40.5076, -74.2654),
    'Plainfield, NJ': (40.6333, -74.4073),
    'Princeton, NJ': (40.3573, -74.6672),
    'Sayreville, NJ': (40.4544, -74.3588),
    'Toms River, NJ': (39.9537, -74.1979),
    'Trenton, NJ': (40.2170, -74.7429),

    '-NEW MEXICO-':(None,None),
    'Alamogordo, NM': (32.8997, -105.8732),
    'Albuquerque, NM': (35.0844, -106.6504),
    'Aztec, NM': (36.8408, -107.9964),
    'Carlsbad, NM': (32.4181, -104.2307),
    'Clovis, NM': (34.4047, -103.2050),
    'Espanola, NM': (35.9910, -106.0750),
    'Hobbs, NM': (32.7125, -103.1445),
    'Las Cruces, NM': (32.3199, -106.7637),
    'Las Vegas, NM': (35.5941, -105.2220),
    'Lovington, NM': (33.3844, -103.3444),
    'Rio Rancho, NM': (35.2321, -106.6630),
    'Santa Fe, NM': (35.6870, -105.9378),
    'Silver City, NM': (32.7783, -108.2772),
    'Taos, NM': (36.4073, -105.5735),
    'Truth or Consequences, NM': (33.1271, -107.2544),
    'West Las Vegas, NM': (35.5924, -105.2320),

    '-NEW YORK-':(None,None),
    'Albany, NY': (42.6526, -73.7562),
    'Angola, NY': (42.6669, -79.0402),
    'Binghamton, NY': (42.0987, -75.9179),
    'Buffalo, NY': (42.8864, -78.8784),
    'Cohoes, NY': (42.7833, -73.7103),
    'Corning, NY': (42.1467, -77.0520),
    'Elmira, NY': (42.0897, -76.8078),
    'Ithaca, NY': (42.4411, -76.5019),
    'Jamaica, NY': (40.6695, -73.7984),
    'Jamestown, NY': (42.0979, -79.2353),
    'Kingston, NY': (41.9276, -73.9970),
    'Lackawanna, NY': (42.8296, -78.8247),
    'Long Beach, NY': (40.5895, -73.6349),
    'Massapequa, NY': (40.6694, -73.4415),
    'Mineola, NY': (40.7459, -73.6406),
    'Montauk, NY': (41.0352, -71.9545),
    'New York City, NY': (40.7128, -74.0060),
    'Ocean Beach, NY': (40.6276, -73.1897),
    'Oyster Bay, NY': (40.8619, -73.5320),
    'Plattsburgh, NY': (44.6995, -73.4533),
    'Port Washington, NY': (40.8068, -73.6486),
    'Roslyn, NY': (40.7834, -73.6392),
    'Rochester, NY': (43.1566, -77.6088),
    'Sand Lake, NY': (42.6530, -73.5795),
    'Saratoga Springs, NY': (43.0831, -73.7837),
    'Schenectady, NY': (42.8142, -73.9396),
    'Sherrill, NY': (43.0713, -75.7583),
    'Syracuse, NY': (43.0481, -76.1474),
    'Troy, NY': (42.7284, -73.6918),
    'Utica, NY': (43.1009, -75.2327),
    'Watertown, NY': (43.9744, -75.9102),
    'West Seneca, NY': (42.8495, -78.7871),

    '-NORTH CAROLINA-':(None,None),
    'Asheville, NC': (35.5951, -82.5515),
    'Boone, NC': (36.2168, -81.6746),
    'Brevard, NC': (35.2337, -82.7343),
    'Burlington, NC': (36.0957, -79.4378),
    'Cary, NC': (35.7915, -78.7811),
    'Chapel Hill, NC': (35.9132, -79.0558),
    'Charlotte, NC': (35.2271, -80.8431),
    'Concord, NC': (35.4088, -80.5795),
    'Durham, NC': (35.9940, -78.8986),
    'Elizabeth City, NC': (36.2946, -76.2510),
    'Fayetteville, NC': (35.0527, -78.8784),
    'Gastonia, NC': (35.2621, -81.1873),
    'Greensboro, NC': (36.0726, -79.7910),
    'Greenville, NC': (35.6127, -77.3664),
    'Hickory, NC': (35.7345, -81.3443),
    'High Point, NC': (35.9557, -80.0053),
    'Jacksonville, NC': (34.7541, -77.4302),
    'New Bern, NC': (35.1085, -77.0441),
    'Raleigh, NC': (35.7796, -78.6382),
    'Rocky Mount, NC': (35.9376, -77.7905),
    'Salisbury, NC': (35.6709, -80.4742),
    'Wilmington, NC': (34.2257, -77.9447),
    'Wilson, NC': (35.7213, -77.9155),
    'Winston-Salem, NC': (36.0999, -80.2442),

    '-NORTH DAKOTA-':(None,None),
    'Bismarck, ND': (46.8083, -100.7837),
    'Devils Lake, ND': (48.1128, -98.8652),
    'Dickinson, ND': (46.8792, -102.7896),
    'Fargo, ND': (46.8772, -96.7898),
    'Grand Forks, ND': (47.9253, -97.0329),
    'Jamestown, ND': (46.9105, -98.7084),
    'Minot, ND': (48.2325, -101.2963),
    'Valley City, ND': (46.9230, -98.0035),
    'Williston, ND': (48.1467, -103.6170),

    '-OHIO-':(None,None),
    'Akron, OH': (41.0814, -81.5190),
    'Canton, OH': (40.7989, -81.3784),
    'Cincinnati, OH': (39.1031, -84.5120),
    'Cleveland, OH': (41.4993, -81.6944),
    'Columbus, OH': (39.9612, -82.9988),
    'Dayton, OH': (39.7589, -84.1916),
    'Dublin, OH': (40.0992, -83.1141),
    'Lima, OH': (40.7426, -84.1052),
    'Mansfield, OH': (40.7584, -82.5154),
    'Marietta, OH': (39.4154, -81.4548),
    'Parma, OH': (41.4048, -81.7229),
    'Springfield, OH': (39.9242, -83.8088),
    'Toledo, OH': (41.6528, -83.5379),
    'Youngstown, OH': (41.0998, -80.6495),
    'Zanesville, OH': (39.9403, -82.0132),

    '-OKLAHOMA-':(None,None),
    'Ardmore, OK': (34.1743, -97.1436),
    'Bartlesville, OK': (36.7473, -95.9808),
    'Broken Arrow, OK': (36.0609, -95.7975),
    'Edmond, OK': (35.6528, -97.4781),
    'Enid, OK': (36.3956, -97.8784),
    'Lawton, OK': (34.6087, -98.3903),
    'Muskogee, OK': (35.7479, -95.3697),
    'Norman, OK': (35.2226, -97.4395),
    'Oklahoma City, OK': (35.4676, -97.5164),
    'Stillwater, OK': (36.1156, -97.0584),
    'Tahlequah, OK': (35.9154, -94.9690),
    'Tulsa, OK': (36.1540, -95.9928),
    'Yukon, OK': (35.5064, -97.7625),

    '-OREGON-':(None,None),
    'Ashland, OR': (42.1946, -122.7095),
    'Bend, OR': (44.0582, -121.3153),
    'Coos Bay, OR': (43.3665, -124.2179),
    'Eugene, OR': (44.0521, -123.0868),
    'Grants Pass, OR': (42.4390, -123.3284),
    'Hood River, OR': (45.7054, -121.5215),
    'Klamath Falls, OR': (42.2249, -121.7817),
    'Medford, OR': (42.3265, -122.8756),
    'Newport, OR': (44.6368, -124.0535),
    'Pendleton, OR': (45.6721, -118.7886),
    'Portland, OR': (45.5152, -122.6784),
    'Salem, OR': (44.9429, -123.0351),
    'The Dalles, OR': (45.5946, -121.1787),

    '-PENNSYLVANIA-':(None,None),
    'Allentown, PA': (40.6084, -75.4902),
    'Altoona, PA': (40.5187, -78.3947),
    'Bethlehem, PA': (40.6259, -75.3705),
    'Carlisle, PA': (40.2019, -77.2003),
    'Chambersburg, PA': (39.9376, -77.6611),
    'Erie, PA': (42.1292, -80.0851),
    'Harrisburg, PA': (40.2737, -76.8844),
    'Hazleton, PA': (40.9584, -75.9747),
    'Johnstown, PA': (40.3267, -78.9212),
    'Lancaster, PA': (40.0379, -76.3055),
    'Philadelphia, PA': (39.9526, -75.1652),
    'Pittsburgh, PA': (40.4406, -79.9959),
    'Reading, PA': (40.3356, -75.9269),
    'Scranton, PA': (41.4089, -75.6624),
    'State College, PA': (40.7934, -77.8600),
    'Wilkes-Barre, PA': (41.2459, -75.8813),
    'York, PA': (39.9626, -76.7277),

    '-RHODE ISLAND-':(None,None),
    'Bristol, RI': (41.6771, -71.2662),
    'Cranston, RI': (41.7798, -71.4373),
    'Newport, RI': (41.4901, -71.3128),
    'Pawtucket, RI': (41.8787, -71.3826),
    'Providence, RI': (41.8240, -71.4128),
    'Warwick, RI': (41.7001, -71.4162),
    'Westerly, RI': (41.3770, -71.8273),
    'Woonsocket, RI': (42.0029, -71.5148),

    '-SOUTH CAROLINA-':(None,None),
    'Aiken, SC': (33.5604, -81.7196),
    'Anderson, SC': (34.5034, -82.6501),
    'Charleston, SC': (32.7765, -79.9311),
    'Columbia, SC': (34.0007, -81.0348),
    'Florence, SC': (34.1954, -79.7626),
    'Greenville, SC': (34.8526, -82.3940),
    'Hilton Head Island, SC': (32.2163, -80.7526),
    'Myrtle Beach, SC': (33.6891, -78.8867),
    'Rock Hill, SC': (34.9249, -81.0251),
    'Spartanburg, SC': (34.9496, -81.9320),
    'Summerville, SC': (33.0185, -80.1756),

    '-SOUTH DAKOTA-':(None,None),
    'Brookings, SD': (44.3114, -96.7984),
    'Deadwood, SD': (44.3766, -103.7296),
    'Huron, SD': (44.3635, -98.2147),
    'Mitchell, SD': (43.7094, -98.0298),
    'Pierre, SD': (44.3683, -100.3510),
    'Rapid City, SD': (44.0805, -103.2310),
    'Sioux Falls, SD': (43.5446, -96.7311),
    'Spearfish, SD': (44.4908, -103.8594),
    'Vermillion, SD': (42.7794, -96.9292),
    'Watertown, SD': (44.8994, -97.1150),
    'Yankton, SD': (42.8711, -97.3973),

    '-TENNESSEE-':(None,None),
    'Chattanooga, TN': (35.0456, -85.3097),
    'Clarksville, TN': (36.5298, -87.3595),
    'Jackson, TN': (35.6145, -88.8139),
    'Johnson City, TN': (36.3134, -82.3535),
    'Knoxville, TN': (35.9606, -83.9207),
    'Memphis, TN': (35.1495, -90.0490),
    'Middleton, TN': (35.5025, -88.7421),
    'Morristown, TN': (36.2134, -83.2945),
    'Nashville, TN': (36.1627, -86.7816),
    'Pigeon Forge, TN': (35.7899, -83.5547),
    'Sevierville, TN': (35.8704, -83.5607),
    'Springfield, TN': (36.5090, -86.8978),
    'Tullahoma, TN': (35.3621, -86.2094),
    'Union City, TN': (36.0317, -88.9180),

    '-TEXAS-':(None,None),
    'Abilene, TX': (32.4487, -99.7331),
    'Amarillo, TX': (35.2210, -101.8313),
    'Austin, TX': (30.2672, -97.7431),
    'Baytown, TX': (29.7355, -94.9771),
    'Beaumont, TX': (30.0802, -94.1266),
    'Brownsville, TX': (25.9017, -97.4975),
    'College Station, TX': (30.6278, -96.3344),
    'Conroe, TX': (30.3119, -95.4567),
    'Corpus Christi, TX': (27.8006, -97.3964),
    'Dallas, TX': (32.7767, -96.7970),
    'El Paso, TX': (31.7619, -106.4850),
    'Fort Worth, TX': (32.7555, -97.3308),
    'Galveston, TX': (29.3013, -94.7977),
    'Garland, TX': (32.9126, -96.6389),
    'Grand Prairie, TX': (32.7459, -96.9978),
    'Houston, TX': (29.7604, -95.3698),
    'Irving, TX': (32.8140, -96.9489),
    'Killeen, TX': (31.1171, -97.7278),
    'Laredo, TX': (27.5064, -99.5075),
    'Lubbock, TX': (33.5779, -101.8552),
    'McKinney, TX': (33.1977, -96.6398),
    'Mesquite, TX': (32.7668, -96.5991),
    'Midland, TX': (31.9970, -102.0779),
    'Odessa, TX': (31.8457, -102.3676),
    'Plano, TX': (33.0198, -96.6989),
    'San Angelo, TX': (31.4638, -100.4370),
    'San Antonio, TX': (29.4241, -98.4936),
    'Temple, TX': (31.0982, -97.3428),
    'Texas City, TX': (29.3839, -94.9018),
    'The Woodlands, TX': (30.1658, -95.4613),
    'Tyler, TX': (32.3512, -95.3011),
    'Waco, TX': (31.5493, -97.1467),
    'Weatherford, TX': (32.7591, -97.8137),
    'Wichita Falls, TX': (33.9137, -98.4936),

    '-UTAH-':(None,None),
    'American Fork, UT': (40.3775, -111.7980),
    'Anderson, UT': (40.4310, -111.7332),
    'Bluffdale, UT': (40.5247, -111.9620),
    'Bountiful, UT': (40.8894, -111.8833),
    'Centerville, UT': (40.8843, -111.8632),
    'Clearfield, UT': (41.1135, -112.0270),
    'Cottonwood Heights, UT': (40.6102, -111.8331),
    'Draper, UT': (40.5247, -111.8632),
    'Eagle Mountain, UT': (40.2970, -111.7599),
    'Farmington, UT': (40.9735, -111.8819),
    'Kaysville, UT': (41.0361, -111.9395),
    'Layton, UT': (41.0616, -111.9707),
    'Lehi, UT': (40.3916, -111.8503),
    'Murray, UT': (40.6662, -111.8877),
    'Orem, UT': (40.2969, -111.6947),
    'Park City, UT': (40.6461, -111.4975),
    'Pleasant Grove, UT': (40.3605, -111.7482),
    'Provo, UT': (40.2338, -111.6585),
    'Sandy, UT': (40.5721, -111.8633),
    'Saratoga Springs, UT': (40.6410, -111.8990),
    'Salt Lake City, UT': (40.7608, -111.8910),
    'Springville, UT': (40.1582, -111.6260),
    'South Salt Lake, UT': (40.7236, -111.8882),
    'Spanish Fork, UT': (40.1113, -111.6547),
    'Taylorsville, UT': (40.6023, -111.9470),
    'West Jordan, UT': (40.6097, -111.9391),
    'West Valley City, UT': (40.6916, -112.0010),

    '-VERMONT-':(None,None),
    'Barre, VT': (44.1990, -72.6982),
    'Barton, VT': (44.5800, -72.2154),
    'Bellows Falls, VT': (43.1303, -72.4500),
    'Bennington, VT': (42.8797, -73.1894),
    'Brattleboro, VT': (42.8555, -72.5578),
    'Burlington, VT': (44.4759, -73.2121),
    'Castleton, VT': (43.6086, -73.2643),
    'Charleston, VT': (44.6104, -71.8735),
    'Colchester, VT': (44.5484, -73.2162),
    'Essex Junction, VT': (44.4983, -73.1151),
    'Hinesburg, VT': (44.3594, -73.0542),
    'Middlebury, VT': (44.0163, -73.1664),
    'Montpelier, VT': (44.2601, -72.5754),
    'Newport, VT': (44.9908, -71.9760),
    'Northfield, VT': (44.1260, -72.6534),
    'Randolph, VT': (43.9797, -72.6420),
    'St. Albans, VT': (44.8006, -73.0810),
    'St. Johnsbury, VT': (44.4190, -71.9791),
    'South Burlington, VT': (44.4741, -73.2150),
    'Vergennes, VT': (44.2017, -73.2585),
    'Winooski, VT': (44.4754, -73.1735),

    '-VIRGINIA-':(None,None),
    'Abingdon, VA': (36.7090, -81.9650),
    'Alexandria, VA': (38.8048, -77.0469),
    'Blacksburg, VA': (37.2296, -80.4139),
    'Bristol, VA': (36.5951, -82.1888),
    'Charlottesville, VA': (38.0293, -78.4767),
    'Chesapeake, VA': (36.7682, -76.2875),
    'Clifton Forge, VA': (37.8139, -79.8122),
    'Colonial Heights, VA': (37.2143, -77.4075),
    'Danville, VA': (36.5859, -79.3950),
    'Fairfax, VA': (38.8462, -77.3064),
    'Falls Church, VA': (38.8833, -77.1536),
    'Franklin, VA': (36.6784, -76.9346),
    'Fredericksburg, VA': (38.3032, -77.4605),
    'Galax, VA': (36.6685, -80.9229),
    'Harrisonburg, VA': (38.4496, -78.8689),
    'Hampton, VA': (37.0299, -76.3452),
    'Hopewell, VA': (37.3044, -77.2898),
    'Leesburg, VA': (39.1157, -77.5636),
    'Lexington, VA': (37.7833, -79.4426),
    'Lynchburg, VA': (37.4138, -79.1423),
    'Manassas, VA': (38.7509, -77.4752),
    'Martinsville, VA': (36.7027, -79.8755),
    'Newport News, VA': (37.0871, -76.4730),
    'Norfolk, VA': (36.8508, -76.2859),
    'Petersburg, VA': (37.2273, -77.4004),
    'Poquoson, VA': (37.1434, -76.3754),
    'Portsmouth, VA': (36.8350, -76.2989),
    'Radford, VA': (37.1314, -80.5762),
    'Richmond, VA': (37.5407, -77.4360),
    'Roanoke, VA': (37.2707, -79.9414),
    'Suffolk, VA': (36.7283, -76.5832),
    'Virginia Beach, VA': (36.8529, -75.9783),
    'Waynesboro, VA': (38.0682, -78.8763),
    'Williamsburg, VA': (37.2707, -76.7075),
    'Winchester, VA': (39.1853, -78.1541),

    '-WASHINGTON-':(None,None),
    'Anacortes, WA': (48.5210, -122.6126),
    'Bellingham, WA': (48.7487, -122.4786),
    'Bremerton, WA': (47.5673, -122.6326),
    'Burien, WA': (47.4742, -122.3482),
    'Centralia, WA': (46.7114, -122.9392),
    'Chelan, WA': (47.8415, -120.0155),
    'Everett, WA': (47.9784, -122.2021),
    'Issaquah, WA': (47.5301, -122.0322),
    'Kennewick, WA': (46.2113, -119.1481),
    'Lakewood, WA': (47.1718, -122.5180),
    'Lacey, WA': (47.0343, -122.8232),
    'Longview, WA': (46.1405, -122.9330),
    'Mount Vernon, WA': (48.4244, -122.3355),
    'Olympia, WA': (47.0379, -122.9007),
    'Port Angeles, WA': (48.1185, -123.4307),
    'Pullman, WA': (46.7310, -117.1804),
    'Richland, WA': (46.2857, -119.2843),
    'Seattle, WA': (47.6062, -122.3321),
    'Spokane, WA': (47.6588, -117.4260),
    'Tacoma, WA': (47.2529, -122.4443),
    'Vancouver, WA': (45.6260, -122.6711),
    'Walla Walla, WA': (46.0622, -118.3403),
    'Yakima, WA': (46.6021, -120.5059),

    '-WEST VIRGINIA-':(None,None),
    'Beckley, WV': (37.7785, -81.1882),
    'Charleston, WV': (38.3498, -81.6326),
    'Clarksburg, WV': (39.2952, -80.3374),
    'Fairmont, WV': (39.4817, -80.1527),
    'Huntington, WV': (38.4192, -82.4451),
    'Martinsburg, WV': (39.4570, -77.9633),
    'Morgantown, WV': (39.6295, -79.9559),
    'Parkersburg, WV': (39.2636, -81.5613),
    'Princeton, WV': (37.3956, -81.1011),
    'Ravenswood, WV': (38.9571, -81.5474),
    'Ripley, WV': (38.8232, -81.7013),
    'Weirton, WV': (40.3923, -80.6223),
    'Wheeling, WV': (40.0657, -80.7200),

    '-WISCONSIN-':(None,None),
    'Appleton, WI': (44.2619, -88.4154),
    'Baraboo, WI': (43.4706, -89.7468),
    'Beaver Dam, WI': (43.4583, -88.8364),
    'Beloit, WI': (42.5083, -89.0318),
    'Brookfield, WI': (43.0607, -88.1049),
    'Green Bay, WI': (44.5133, -88.0133),
    'Iola, WI': (44.3232, -89.1941),
    'Janesville, WI': (42.6820, -89.0187),
    'Kenosha, WI': (42.5803, -87.8212),
    'La Crosse, WI': (43.8001, -91.2396),
    'Madison, WI': (43.0731, -89.4012),
    'Manitowoc, WI': (44.0916, -87.6572),
    'Milwaukee, WI': (43.0389, -87.9065),
    'Oshkosh, WI': (44.0241, -88.5426),
    'Racine, WI': (42.7260, -87.7823),
    'Shawano, WI': (44.7815, -88.5955),
    'Sheboygan, WI': (43.7500, -87.7147),
    'Waukesha, WI': (43.0157, -88.2303),
    'West Bend, WI': (43.4251, -88.1849),
    'Wisconsin Rapids, WI': (44.3911, -89.8257),

    '-WYOMING-':(None,None),
    'Casper, WY': (42.8497, -106.3131),
    'Cheyenne, WY': (41.1400, -104.8202),
    'Gillette, WY': (44.2910, -105.5018),
    'Laramie, WY': (41.3113, -105.5911),
    'Rock Springs, WY': (41.5872, -109.2030),
    'Sheridan, WY': (44.7972, -106.9652),
    'Green River, WY': (41.5140, -108.5400),
    'Jackson, WY': (43.4799, -110.7624),
    'Pinedale, WY': (42.8664, -109.8600),
    'Riverton, WY': (43.0210, -108.3836),

    '-TERRITORIES':(None,None),
    'Pago Pago, American Samoa': (14.2756, -170.7020),
    'Fagatogo, American Samoa': (14.2792, -170.6930),
    'Hagåtña, Guam': (13.4741, 144.7447),
    'Dededo, Guam': (13.5043, 144.8506),
    'Mangilao, Guam': (13.4519, 144.8011),
    'Saipan, Mariana Islands': (15.1580, 145.7500),
    'Garapan, Mariana Islands': (15.2130, 145.7542),
    'San Juan, Puerto Rico': (18.4655, -66.1057),
    'Bayamón, Puerto Rico': (18.4151, -66.1706),
    'Ponce, Puerto Rico': (17.9785, -66.9732),
    'Mayaguez, Puerto Rico': (18.2013, -67.1378),
    'Charlotte Amalie, American Virgin Islands': (18.3400, -64.9307),
    'Cruz Bay, American Virgin Islands': (18.3205, -64.7983),
    'Frederiksted, American Virgin Islands': (17.7117, -64.8885),
}

# Organize cities by state
states = {
    'Alabama': ['Birmingham, AL', 'Dothan, AL', 'Florence, AL', 'Gadsden, AL', 'Hoover, AL', 'Huntsville, AL', 
                'Mobile, AL', 'Montgomery, AL', 'Selma, AL', 'Tuscaloosa, AL'],
    
    'Alaska': ['Anchorage, AK', 'Barrow (Utqiaġvik), AK', 'Bethel, AK', 'Fairbanks, AK', 'Homer, AK', 'Juneau, AK', 
               'Kenai, AK','Ketchikan, AK','Kodiak, AK','Nome, AK','Palmer, AK',
               'Seward, AK','Sitka, AK','Soldotna, AK','Valdez, AK','Wasilla, AK','Wrangell, AK'],
    
    'Arizona': ['Chandler, AZ', 'Flagstaff, AZ','Gilbert, AZ','Glendale, AZ','Mesa, AZ',
                'Phoenix, AZ','Prescott, AZ','Scottsdale, AZ','Tempe, AZ','Tuscon, AZ','Yuma, AZ'],
    
    'California': ['Anaheim, CA','Bakersfield, CA','Berkeley, CA','Bridgeport, CA','Death Valley, CA','El Centro, CA',
                   'Eureka, CA','Fresno, CA','Independence, CA','Long Beach, CA','Los Angeles, CA','Oakland, CA','Redding, CA',
                   'Sacramento, CA','San Diego, CA','San Francisco, CA','San Jose, CA','Santa Barbara, CA','Stockton, CA','Yreka, CA'],


    'Colorado': ['Aspen, CO', 'Boulder, CO', 'Colorado Springs, CO', 'Denver, CO', 'Durango, CO', 
                 'Fort Collins, CO', 'Grand Junction, CO',
                 'Gunnison, CO', 'Pueblo, CO', 'Steamboat Springs, CO', 'Vail, CO'],
    
    'Connecticut': ['Bridgeport, CT', 'Danbury, CT', 'Hartford, CT', 'New Haven, CT', 'New London, CT', 
                    'Norwalk, CT', 'Stamford, CT', 'Waterbury, CT'],
    
    'Delaware': ['Dover, DE', 'Middletown, DE', 'New Castle, DE', 'Wilmington, DE'],

    'Florida':['Clearwater, FL', 'Fort Lauderdale, FL', 'Fort Myers, FL', 'Gainesville, FL',
               'Jacksonville, FL', 'Miami, FL', 'Naples, FL', 'Orlando, FL', 'Pensacola, FL',
               'St. Petersburg, FL', 'Tampa, FL', 'West Palm Beach, FL'],
    
    'Georgia':['Atlanta, GA', 'Athens, GA', 'Augusta, GA', 'Columbus, GA', 'Macon, GA', 
               'Marietta, GA', 'Roswell, GA', 'Savannah, GA', 'Smyrna, GA', 'Valdosta, GA'],

    'Hawaii': ['Hilo, HI','Honolulu, HI', 'Kailua, HI', 'Kaneohe, HI', 'Lihue, HI', 
               'Makakilo, HI', 'Pearl City, HI', 'Waianae, HI'],

    'Idaho': ['Boise, ID', 'Coeur d\'Alene, ID', 'Idaho Falls, ID', 'Moscow, ID', 'Nampa, ID',
              'Pocatello, ID', 'Twin Falls, ID', 'Rexburg, ID', 'Meridian, ID', 'Post Falls, ID'],

    'Illinois': ['Aurora, IL', 'Bloomington, IL', 'Carbondale, IL', 'Champaign, IL', 'Chicago, IL', 'Decatur, IL', 
                 'Elgin, IL', 'Joliet, IL', 'Naperville, IL', 'Peoria, IL', 'Rockford, IL', 'Skokie, IL', 'Springfield, IL', 
                 'Waukegan, IL'],
    
    'Indiana': ['Anderson, IN', 'Bloomington, IN', 'Charlestown, IN', 'Columbus, IN', 'Evansville, IN', 'Fort Wayne, IN', 'Gary, IN',
                'Indianapolis, IN', 'Kokomo, IN', 'Lafayette, IN', 'Muncie, IN', 
                'Noblesville, IN', 'South Bend, IN', 'Terre Haute, IN', 'West Lafayette, IN'],

    'Iowa': ['Ankeny, IA', 'Bettendorf, IA', 'Council Bluffs, IA', 'Des Moines, IA', 'Dubuque, IA', 
             'Iowa City, IA', 'Marshalltown, IA', 'Mason City, IA', 'Newton, IA', 'Sioux City, IA', 
             'Waterloo, IA', 'West Des Moines, IA'],

    'Kentucky': ['Ashland, KY', 'Bowling Green, KY', 'Covington, KY', 'Elizabethtown, KY', 'Florence, KY', 
                 'Frankfort, KY', 'Hopkinsville, KY', 'Lexington, KY', 'Louisville, KY', 'Madisonville, KY',
                 'Monticello, KY', 'Owensboro, KY', 'Paducah, KY', 'Richmond, KY', 'Somerset, KY', 'Walton, KY'],

    'Louisiana': ['Abbeville, LA', 'Alexandria, LA', 'Baton Rouge, LA', 'Bossier City, LA', 'Covington, LA',
                  'Denham Springs, LA', 'Houma, LA', 'La Place, LA', 'Lafayette, LA', 'Lake Charles, LA', 'Mandeville, LA', 
                  'Metairie, LA', 'Monroe, LA', 'New Iberia, LA', 'New Orleans, LA', 'Shreveport, LA', 'Slidell, LA', 'West Monroe, LA'],

    'Maine': ['Augusta, ME', 'Bangor, ME', 'Bath, ME', 'Biddeford, ME', 'Brunswick, ME', 'Caribou, ME', 
              'Lewiston, ME', 'Portland, ME', 'Saco, ME', 'Sanford, ME', 'Waterville, ME'],

    'Maryland': ['Annapolis, MD', 'Baltimore, MD', 'Bethesda, MD', 'Bowie, MD', 'College Park, MD', 'Frederick, MD', 
                 'Hagerstown, MD', 'Laurel, MD', 'Piney Point, MD', 'Rockville, MD', 'Silver Spring, MD', 'Takoma Park, MD'],

    'Massachusettes': ['Boston, MA', 'Brockton, MA', 'Cambridge, MA', 'Cape Cod, MA', 'Fall River, MA',
                       'Framingham, MA', 'Gloucester, MA', 'Lawrence, MA', 'Lynn, MA', 'Lowell, MA', 
                       'New Bedford, MA', 'North Adams, MA', 'Pittsfield, MA', 'Quincy, MA', 'Springfield, MA', 'Worcester, MA'],

    'Michigan': ['Ann Arbor, MI', 'Battle Creek, MI', 'Bay City, MI', 'Detroit, MI', 'Flint, MI', 'Grand Blanc, MI', 'Grand Rapids, MI', 
                 'Holland, MI', 'Kalamazoo, MI', 'Lansing, MI', 'Livonia, MI', 'Muskegon, MI', 'Pontiac, MI', 'Saginaw, MI', 'Sterling Heights, MI', 
                 'Troy, MI', 'Waterford, MI', 'Westland, MI', 'Wyoming, MI'],

    'Mississippi':['Biloxi, MS', 'Brandon, MS', 'Gulfport, MS', 'Hattiesburg, MS', 'Jackson, MS', 'Laurel, MS', 
                   'Meridian, MS', 'Mississippi City, MS', 
                   'Oxford, MS', 'Pearl, MS', 'Picayune, MS', 'Southaven, MS', 'Vicksburg, MS'],

    'Missouri':['Columbia, MO', 'Independence, MO', 'Kansas City, MO', 'Lee\'s Summit, MO', 
                'St. Charles, MO', 'St. Louis, MO', 'Saint Joseph, MO', 'Springfield, MO'],

    'Montana': ['Billings, MT', 'Bozeman, MT', 'Butte, MT', 'Cut Bank, MT', 'Glendive, MT', 
                'Great Falls, MT', 'Havre, MT', 'Helena, MT', 'Kalispell, MT', 'Lewistown, MT', 
                'Livingston, MT', 'Miles City, MT', 'Missoula, MT', 'Red Lodge, MT'],

    'Nebraska': ['Beatrice, NE', 'Bellevue, NE', 'Columbus, NE', 'Fremont, NE', 'Grand Island, NE', 
                 'Kearney, NE', 'Lincoln, NE', 'Norfolk, NE', 'Omaha, NE', 'Scottsbluff, NE', 'York, NE'],

    'Nevada': ['Boulder City, NV', 'Carson City, NV', 'Elko, NV', 'Henderson, NV', 'Las Vegas, NV', 
               'Lincoln, NV', 'Reno, NV', 'Sparks, NV', 'Sun Valley, NV', 'West Wendover, NV'],

    'New Hampshire': ['Concord, NH', 'Durham, NH', 'Hampton, NH', 'Keene, NH', 'Laconia, NH', 
                      'Lebanon, NH','Merrimack, NH', 'Manchester, NH','Mt. Washington, NH', 'Nashua, NH', 'Portsmouth, NH'],

    'New Jersey': ['Atlantic City, NJ', 'Bayonne, NJ', 'Camden, NJ', 'East Orange, NJ', 'Elizabeth, NJ', 
                   'Ewing, NJ', 'Hackensack, NJ', 'Hoboken, NJ', 'Jersey City, NJ', 'New Brunswick, NJ', 
                   'Newark, NJ', 'Passaic, NJ', 'Paterson, NJ', 'Perth Amboy, NJ', 'Plainfield, NJ', 'Princeton, NJ',
                   'Sayreville, NJ', 'Toms River, NJ', 'Trenton, NJ'],

    'New Mexico': ['Alamogordo, NM', 'Albuquerque, NM', 'Aztec, NM', 'Carlsbad, NM', 'Clovis, NM', 'Espanola, NM', 
                   'Hobbs, NM', 'Las Cruces, NM', 'Las Vegas, NM', 'Lovington, NM', 'Rio Rancho, NM', 'Santa Fe, NM', 
                   'Silver City, NM', 'Taos, NM', 'Truth or Consequences, NM', 'West Las Vegas, NM'],

    'New York': ['Albany, NY', 'Angola, NY', 'Binghamton, NY', 'Buffalo, NY', 'Cohoes, NY', 'Corning, NY', 'Elmira, NY', 'Ithaca, NY', 'Jamaica, NY', 
                 'Jamestown, NY', 'Kingston, NY', 'Lackawanna, NY', 'Long Beach, NY', 'Massapequa, NY', 'Mineola, NY', 'Montauk, NY', 'New York City, NY', 
                 'Ocean Beach, NY', 'Oyster Bay, NY', 'Plattsburgh, NY', 'Port Washington, NY', 'Roslyn, NY', 'Rochester, NY', 'Sand Lake, NY', 'Saratoga Springs, NY', 
                 'Schenectady, NY', 'Sherrill, NY', 'Syracuse, NY', 'Troy, NY', 'Utica, NY', 'Watertown, NY', 'West Seneca, NY'],

    'North Carolina': ['Asheville, NC', 'Boone, NC', 'Brevard, NC', 'Burlington, NC', 'Cary, NC', 'Chapel Hill, NC', 'Charlotte, NC', 'Concord, NC', 'Durham, NC', 
                       'Elizabeth City, NC', 'Fayetteville, NC', 'Gastonia, NC', 'Greensboro, NC', 'Greenville, NC', 'Hickory, NC', 'High Point, NC', 'Jacksonville, NC', 
                       'New Bern, NC', 'Raleigh, NC', 'Rocky Mount, NC', 'Salisbury, NC', 'Wilmington, NC', 'Wilson, NC', 'Winston-Salem, NC'],
    
    'North Dakota': ['Bismarck, ND', 'Devils Lake, ND', 'Dickinson, ND', 'Fargo, ND', 'Grand Forks, ND', 'Jamestown, ND', 
                     'Minot, ND', 'Valley City, ND', 'Williston, ND'],

    'Ohio':['Akron, OH', 'Canton, OH', 'Cincinnati, OH', 'Cleveland, OH', 'Columbus, OH', 'Dayton, OH', 'Dublin, OH', 
            'Lima, OH', 'Mansfield, OH', 'Marietta, OH', 'Parma, OH', 'Springfield, OH', 'Toledo, OH', 'Youngstown, OH', 'Zanesville, OH'],

    'Oklahoma':['Ardmore, OK', 'Bartlesville, OK', 'Broken Arrow, OK', 'Edmond, OK', 'Enid, OK', 'Lawton, OK', 'Muskogee, OK',
                'Norman, OK', 'Oklahoma City, OK', 'Stillwater, OK', 'Tahlequah, OK', 'Tulsa, OK', 'Yukon, OK'],

    'Oregon':['Ashland, OR', 'Bend, OR', 'Coos Bay, OR', 'Eugene, OR', 'Grants Pass, OR', 'Hood River, OR',
              'Klamath Falls, OR', 'Medford, OR', 'Newport, OR', 'Pendleton, OR', 'Portland, OR', 'Salem, OR', 'The Dalles, OR'],

    'Pennsylvania':['Allentown, PA', 'Altoona, PA', 'Bethlehem, PA', 'Carlisle, PA', 'Chambersburg, PA', 'Erie, PA', 'Harrisburg, PA', 'Hazleton, PA', 
                    'Johnstown, PA', 'Lancaster, PA', 'Philadelphia, PA', 'Pittsburgh, PA', 'Reading, PA', 'Scranton, PA', 
                    'State College, PA', 'Wilkes-Barre, PA', 'York, PA'],

    'Rhode Island':['Bristol, RI', 'Cranston, RI', 'Newport, RI', 'Pawtucket, RI', 
                    'Providence, RI', 'Warwick, RI', 'Westerly, RI', 'Woonsocket, RI'],

    'South Carolina':['Aiken, SC', 'Anderson, SC', 'Charleston, SC', 'Columbia, SC', 'Florence, SC', 
                      'Greenville, SC', 'Hilton Head Island, SC', 'Myrtle Beach, SC', 'Rock Hill, SC',
                      'Spartanburg, SC', 'Summerville, SC'],

    'South Dakota':['Brookings, SD', 'Deadwood, SD', 'Huron, SD', 'Mitchell, SD', 'Pierre, SD', 'Rapid City, SD', 
                     'Sioux Falls, SD', 'Spearfish, SD', 'Vermillion, SD', 'Watertown, SD', 'Yankton, SD'],

    'Tennessee':['Chattanooga, TN', 'Clarksville, TN', 'Jackson, TN', 'Johnson City, TN', 'Knoxville, TN', 'Memphis, TN',
                 'Middleton, TN', 'Morristown, TN', 'Nashville, TN', 'Pigeon Forge, TN', 'Sevierville, TN', 'Springfield, TN',
                 'Tullahoma, TN', 'Union City, TN'],

    'Texas':['Abilene, TX', 'Amarillo, TX', 'Austin, TX', 'Baytown, TX', 'Beaumont, TX', 'Brownsville, TX', 'College Station, TX','Corpus Christi, TX',
             'Conroe, TX', 'Dallas, TX', 'El Paso, TX', 'Fort Worth, TX', 'Galveston, TX', 'Garland, TX', 'Grand Prairie, TX', 'Houston, TX', 'Irving, TX',
             'Killeen, TX', 'Laredo, TX', 'Lubbock, TX', 'McKinney, TX', 'Mesquite, TX', 'Midland, TX', 'Odessa, TX', 'Plano, TX', 'San Angelo, TX',
             'San Antonio, TX', 'Temple, TX', 'Texas City, TX', 'The Woodlands, TX', 'Tyler, TX', 'Waco, TX', 'Weatherford, TX', 'Wichita Falls, TX'],

    'Utah':['American Fork, UT', 'Anderson, UT', 'Bluffdale, UT', 'Bountiful, UT', 'Centerville, UT', 'Clearfield, UT', 
            'Cottonwood Heights, UT', 'Draper, UT', 'Eagle Mountain, UT', 'Farmington, UT', 'Kaysville, UT', 'Layton, UT', 
            'Lehi, UT', 'Murray, UT', 'Orem, UT', 'Park City, UT', 'Pleasant Grove, UT', 'Provo, UT', 'Sandy, UT', 'Saratoga Springs, UT', 
            'Salt Lake City, UT', 'Springville, UT', 'South Salt Lake, UT', 'Spanish Fork, UT', 'Taylorsville, UT', 'West Jordan, UT', 'West Valley City, UT'],

    'Vermont':['Barre, VT', 'Barton, VT', 'Bellows Falls, VT', 'Bennington, VT', 'Brattleboro, VT', 'Burlington, VT', 'Castleton, VT', 'Charleston, VT',
               'Colchester, VT', 'Essex Junction, VT', 'Hinesburg, VT', 'Middlebury, VT', 'Montpelier, VT', 'Newport, VT', 'Northfield, VT', 'Randolph, VT', 
               'St. Albans, VT', 'St. Johnsbury, VT', 'South Burlington, VT', 'Vergennes, VT', 'Winooski, VT'],

    'Virginia':['Abingdon, VA', 'Alexandria, VA', 'Blacksburg, VA', 'Bristol, VA', 'Charlottesville, VA', 'Chesapeake, VA', 'Clifton Forge, VA', 
                'Colonial Heights, VA', 'Danville, VA', 'Fairfax, VA', 'Falls Church, VA', 'Franklin, VA', 'Fredericksburg, VA', 'Galax, VA', 
                'Harrisonburg, VA', 'Hampton, VA', 'Hopewell, VA', 'Leesburg, VA', 'Lexington, VA', 'Lynchburg, VA', 'Manassas, VA', 'Martinsville, VA',
                'Newport News, VA', 'Norfolk, VA', 'Petersburg, VA', 'Poquoson, VA', 'Portsmouth, VA', 'Radford, VA', 'Richmond, VA', 'Roanoke, VA', 
                'Suffolk, VA', 'Virginia Beach, VA', 'Waynesboro, VA', 'Williamsburg, VA', 'Winchester, VA'],

    'Washington':['Anacortes, WA', 'Bellingham, WA', 'Bremerton, WA', 'Burien, WA', 'Centralia, WA', 'Chelan, WA', 'Everett, WA', 'Issaquah, WA',
                  'Kennewick, WA', 'Lakewood, WA', 'Lacey, WA', 'Longview, WA', 'Mount Vernon, WA', 'Olympia, WA', 'Port Angeles, WA', 'Pullman, WA',
                  'Richland, WA', 'Seattle, WA', 'Spokane, WA', 'Tacoma, WA', 'Vancouver, WA', 'Walla Walla, WA', 'Yakima, WA'],

    'West Virginia':['Beckley, WV', 'Charleston, WV', 'Clarksburg, WV', 'Fairmont, WV', 'Huntington, WV', 'Martinsburg, WV', 
                     'Morgantown, WV', 'Parkersburg, WV', 'Princeton, WV', 'Ravenswood, WV', 'Ripley, WV', 'Weirton, WV', 'Wheeling, WV'],

    'Wisconsin':['Appleton, WI', 'Baraboo, WI', 'Beaver Dam, WI', 'Beloit, WI', 'Brookfield, WI', 'Green Bay, WI', 'Iola, WI',
                 'Janesville, WI', 'Kenosha, WI', 'La Crosse, WI', 'Madison, WI', 'Manitowoc, WI', 'Milwaukee, WI', 'Oshkosh, WI', 
                 'Racine, WI', 'Shawano, WI', 'Sheboygan, WI', 'Waukesha, WI', 'West Bend, WI', 'Wisconsin Rapids, WI'],

    'Wyoming':['Casper, WY', 'Cheyenne, WY', 'Gillette, WY', 'Laramie, WY', 'Rock Springs, WY', 'Sheridan, WY',
               'Green River, WY', 'Jackson, WY', 'Pinedale, WY', 'Riverton, WY'],

    'Territories':['Pago Pago, American Samoa', 'Fagatogo, American Samoa', 'Hagåtña, Guam', 'Dededo, Guam', 'Mangilao, Guam',
                   'Saipan, Mariana Islands ', 'Garapan, Mariana Islands ', 'San Juan, Puerto Rico', 'Bayamón, Puerto Rico', 'Ponce, Puerto Rico', 'Mayaguez, Puerto Rico', 
                   'Charlotte Amalie, American Virgin Islands', 'Cruz Bay, American Virgin Islands', 'Frederiksted, American Virgin Islands']

    

    

    


    

    
    

    

}

# Dropdown for state selection
state_select = pn.widgets.Select(name='Select State', options=list(states.keys()))

# Dropdown for city selection (based on the selected state)
city_select = pn.widgets.Select(name='Select City', options=[])

# Text input widgets for coordinates
lattxt = pn.widgets.TextInput(name='Enter Latitude Coordinate', placeholder='-90 to 90')
lontxt = pn.widgets.TextInput(name='Enter Longitude Coordinate', placeholder='-180 to 180')

# Widget to display messages
output = pn.widgets.StaticText(value="", width=400)

def update_city_options(event):
    """Update city options based on selected state."""
    selected_state = event.new
    if selected_state:
        city_select.options = states[selected_state]
    else:
        city_select.options = []

state_select.param.watch(update_city_options, 'value')

def update_coords(event):
    """Update latitude and longitude fields based on selected city."""
    selected_city = event.new
    if selected_city:
        lat, lon = city_coords[selected_city]
        lattxt.value = str(lat)
        lontxt.value = str(lon)
        output.value = f"Coordinates for {selected_city} selected!"
    else:
        lattxt.value = ""
        lontxt.value = ""
        output.value = "No city selected. Enter coordinates manually."

city_select.param.watch(update_coords, 'value')

def check_coords(event):
    """Validate latitude and longitude inputs."""
    output.value = ""
    
    try:
        # Check Longitude
        if event.obj == lontxt:
            if lontxt.value == '':
                return 
            value = float(lontxt.value)
            if value < -180 or value > 180:
                raise ValueError("WARNING!: Longitude input out of range.")
            else:
                output.value = "Success! Input Longitude"
        
        # Check Latitude
        elif event.obj == lattxt:
            if lattxt.value == '':
                return  
            value = float(lattxt.value)
            if value < -90 or value > 90:
                raise ValueError("WARNING!: Latitude input out of range.")
            else:
                output.value = "Success! Input Latitude"
        
    except ValueError as e:
        output.value = f"Error occurred: {e}"
    except Exception as e:
        output.value = f"An unexpected error occurred: {e}"

# Watch for changes in the text inputs
lontxt.param.watch(check_coords, 'value')
lattxt.param.watch(check_coords, 'value')

# Create layout
layout = pn.Column(state_select, city_select, lattxt, lontxt, output)
layout.servable()

## **Run cell to display the SounderPy style sounding**
***

In [ ]:
def get_era5_soundings(dateList, lattxt, lontxt, spy):
    split_date = dateList[0].split(',')
    day = int(split_date[1])
    month = int(split_date[2])
    year =  int(split_date[0])
    hour = int(split_date[3])

    latitude = float(lattxt.value)
    longitude = float(lontxt.value)
    
    cleandata=spy.get_model_data('era5',[latitude, longitude], year, month, day, hour)
    spy.build_sounding(cleandata)


for time in dateList:
    get_era5_soundings(dateList,lattxt,lontxt,spy)
